In [1]:
import numpy as np
import pandas as pd
import torch
torch.set_default_tensor_type(torch.FloatTensor) 
import copy
import sys
import os
notebook_dir = os.getcwd()
parent_dir = os.path.dirname(notebook_dir)
# Add the parent directory to the Python path
sys.path.append(parent_dir)

from rct_data_generator import *
from outcome_models import *
from plotting_functions import *
from mcmc_bayes_update import *
from eig_comp_utils import *
from research_exp_utils import *
import uci_dataset as dataset

from causalml.inference.tree import CausalForestDML

/Users/lucile/causal_info_gain/pjake/lib/python3.9/site-packages/torch/__init__.py:696: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/tensor/python_tensor.cpp:453.)
  _C._set_default_tensor_type(t)


In [2]:
initial_data = dataset.load_abalone()
initial_data['Sex'] = initial_data['Sex'].map({'M': 0, 'F': 1})
initial_data.dropna(inplace=True)

In [54]:
number_of_candidate_sites = 10
min_sample_size_cand = 150
max_sample_size_cand = 250
host_sample_size = 200
desired_initial_sample_size = 10**6
initial_data = initial_data.sample(n=desired_initial_sample_size, replace=True, random_state=42)


outcome_function = lambda X, T, eps: 1 + 1 * X['Sex'] - 1 * X['Weight.viscera'] + np.log(X['Weight.whole']) - \
    X['Height'] + 4 * T + 2* X['Weight.shucked']*T + 24* X['Weight.shell']*T + 0* X['Weight.shucked']*T + eps 
std_true_y = 1
power_x = 1
power_x_t = 1
sigma_rand_error = 1

exp_parameters = {'number_of_candidate_sites': number_of_candidate_sites, 'min_sample_size_cand': min_sample_size_cand, \
                'max_sample_size_cand': max_sample_size_cand, 'host_sample_size': host_sample_size, 'outcome_function': outcome_function, \
                'std_true_y': std_true_y, 'power_x': power_x, 'power_x_t': power_x_t}

causal_param_first_index = causal_param_first_index = power_x * np.shape(initial_data)[1] + 1   

In [62]:
def generating_random_sites_from(data, exp_parameters):
    sites = {}
    sample_size, number_covariates = np.shape(data)[0], np.shape(data)[1]
    function_indices = {0: lambda X: np.log(X+1), 1: lambda X: X**3, 2: lambda X: X, 3: lambda X: X**2}
    T = np.random.randint(2, size=sample_size)
    number_of_candidate_sites = exp_parameters['number_of_candidate_sites']
    min_sample_size_cand = exp_parameters['min_sample_size_cand']
    max_sample_size_cand = exp_parameters['max_sample_size_cand']
    outcome_function = exp_parameters['outcome_function']
    std_true_y = exp_parameters['std_true_y']
    power_x = exp_parameters['power_x']
    power_x_t = exp_parameters['power_x_t']
    number_features = number_covariates + 1
    created_sites = 0
    
    while created_sites < number_of_candidate_sites:
        
        selected_features_for_subsampling = np.random.randint(2, size = number_features) 
        # binary bool vector representing selection for being an input of the sampling function
        random_coefs = [np.random.uniform(-10, 10) for _ in range(number_features)] 
        random_fct_idx = [np.random.randint(0, 4) for _ in range(number_features)] 
        
        def p_assigned_to_site(X, T, eps):
            result = 0
            for j in range(number_features-1):
                result += selected_features_for_subsampling[j] * random_coefs[j] * function_indices[random_fct_idx[j]](X[j])
            result += selected_features_for_subsampling[-1] * random_coefs[-1] *  function_indices[random_fct_idx[-1]](T)
            return sigmoid(result + eps)
        
        sample_size = np.random.randint(min_sample_size_cand, max_sample_size_cand + 1)  # Add 1 to include max_sample_size_cand
        design_data_cand = subsample_one_dataset(data, T, p_assigned_to_site, sample_size, power_x, power_x_t, outcome_function, std_true_y)
        if not design_data_cand.empty:
            sites[created_sites] = design_data_cand
            created_sites += 1

    return sites

In [63]:
#dictionnary of random sites
candidate_sites = generating_random_sites_from(initial_data, exp_parameters)
host = candidate_sites.popitem()[1]

# Prior parameters for Bayesian update on host
d = np.shape(host)[1] -1
prior_mean = torch.zeros(d)
beta_0, sigma_0_sq, inv_cov_0 = prior_mean, sigma_rand_error,torch.eye(d)
prior_hyperparameters = {'beta_0': beta_0, 'sigma_0_sq': sigma_0_sq,"inv_cov_0":inv_cov_0}


/Users/lucile/causal_info_gain/causal_prospective_merge/rct_data_generator.py:19: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Sample size of subsampled dataset is0 which is not equal to the wanted sample size of 184)


/Users/lucile/causal_info_gain/causal_prospective_merge/rct_data_generator.py:19: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Sample size of subsampled dataset is0 which is not equal to the wanted sample size of 250)


In [57]:
X_host, Y_host = torch.from_numpy(host.drop(columns=["Y"]).values), torch.from_numpy(host["Y"].values)
n_samples_outer_expectation = 20
n_samples_inner_expectation = 30
results = {}

In [58]:
results = {"EIG_obs_from_samples": [], 'EIG_caus_from_samples':[], "EIG_obs_closed_form":[], "EIG_caus_closed_form":[]}

In [59]:
for _,candidate in candidate_sites.items():
    X_cand = torch.from_numpy(candidate.drop(columns=["Y"]).values)
    bayes_reg = BayesianLinearRegression(prior_hyperparameters)
    bayes_reg.set_causal_index(causal_param_first_index)
    post_host_parameters = bayes_reg.fit(X_host, Y_host)
    n_samples = n_samples_outer_expectation * (n_samples_inner_expectation + 1)

    results["EIG_obs_from_samples"].append(
            bayes_reg.samples_obs_EIG(
                X_cand, n_samples_outer_expectation, n_samples_inner_expectation
            )
        )
    results["EIG_caus_from_samples"].append(
            bayes_reg.samples_causal_EIG(
                X_cand, n_samples_outer_expectation, n_samples_inner_expectation
            )
        )

In [60]:
for _, candidate in candidate_sites.items():
    X_cand = torch.from_numpy(candidate.drop(columns=["Y"]).values)
    bayes_reg = BayesianLinearRegression(prior_hyperparameters)
    bayes_reg.set_causal_index(causal_param_first_index)
    post_host_parameters = bayes_reg.fit(X_host, Y_host)
    n_samples = n_samples_outer_expectation * (n_samples_inner_expectation + 1)

    results["EIG_obs_closed_form"].append(
            bayes_reg.closed_form_obs_EIG(X_cand)
            )
    results["EIG_caus_closed_form"].append(
            bayes_reg.closed_form_causal_EIG(X_cand)
            )

In [ ]:
# now merge and compute some CATE error
merged_datasets = {}

for i, candidate in candidate_sites.items():
    merged_datasets[i]= pd.concat([host, candidate], axis=0)

In [73]:
CATE_errors = {}
merged_datasets = {}

from econml.metalearners import TLearner
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
model_y = GradientBoostingRegressor()
model_t = GradientBoostingClassifier()

for i, candidate in merged_datasets.items():
    merged_datasets[i]= pd.concat([host, candidate], axis=0)
    X_merged = merged_datasets[i].filter(regex='^(?!T)').copy()
    X_merged = X_merged.drop(columns=["Y"])
    T_merged = merged_datasets[i]['T']
    Y_merged = merged_datasets[i]['Y']

    learner = TLearner(model_y=model_y, model_t=model_t)
    learner.fit(Y=Y_merged, T=T_merged, X=X_merged)
    cate = learner.effect(X_merged)
    ### need dataset with ground truth to compute some kind of errors here
